# Determing number of hackers involved

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Hacker Identification').getOrCreate()

In [2]:
data = spark.read.csv('hack_data.csv', header=True,inferSchema=True)

In [3]:
data.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [4]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [5]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [6]:
assembler = VectorAssembler(
    inputCols = ['Session_Connection_Time', 'Bytes Transferred',
                 'Kali_Trace_Used', 'Servers_Corrupted',
                 'Pages_Corrupted', 'WPM_Typing_Speed'],
    outputCol = 'features')

In [7]:
vectordata = assembler.transform(data)

In [8]:
vectordata.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [9]:
from pyspark.ml.feature import StandardScaler
scalar = StandardScaler(inputCol = 'features', outputCol = 'ScaledFeatures')

In [10]:
mldata = scalar.fit(vectordata).transform(vectordata)

In [11]:
mldata.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- ScaledFeatures: vector (nullable = true)



In [12]:
finaldata = mldata.select('scaledFeatures')
finaldata.printSchema()

root
 |-- scaledFeatures: vector (nullable = true)



In [13]:
kmeans2 = KMeans(featuresCol='scaledFeatures', k=2)
kmeans3 = KMeans(featuresCol='scaledFeatures', k=3)

In [14]:
model_k2 = kmeans2.fit(finaldata)
model_k3 = kmeans3.fit(finaldata)

Now, checking out that how many hacker does this two hacker or three hackers on the basis of that the forensis people can make their idntification that wheather or not the third hacker is involved or not

In [15]:
model_k3.transform(finaldata).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   88|
|         0|   79|
+----------+-----+



This won't sheem like the condition describe by them that their attacks are somewhat related. This makes clear that the third hacker is not involved in. 

In [16]:
model_k2.transform(finaldata).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



This makes things clear that only the hacker 1 and hacker 2 are involved.